<a href="https://colab.research.google.com/github/therealhaseeb/MyThesis/blob/main/1_GenomicPipelineForThesis(Hail%2BPandas_Dask).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Author: HASEEB ZAHID

In [ ]:
!pip install --upgrade urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!curl -O ftp://ngs.sanger.ac.uk/production/hgdp/hgdp_wgs.20190516/hgdp_wgs.20190516.full.chrX.vcf.gz
!curl -O ftp://ngs.sanger.ac.uk/production/hgdp/hgdp_wgs.20190516/hgdp_wgs.20190516.full.chrX.vcf.gz.tbi

In [ ]:
!apt-get install -y \
    openjdk-8-jre-headless \
    g++ \
    python3.6 python3-pip \
    libopenblas-base liblapack3

!pip install hail

In [ ]:
mt = hl.import_vcf('/content/hgdp_wgs.20190516.full.chrX.vcf.gz', reference_genome='GRCh38', force_bgz=True)
print(mt.count())

Initializing Hail with default parameters...
Running on Apache Spark version 3.1.3
SparkUI available at http://78fac6c4aa73:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /content/hail-20220725-1554-0.2.97-937922d7f46c.log
2022-07-25 15:54:53 Hail: INFO: Coerced sorted dataset


(132457, 610)


In [ ]:
import hail as hl
from hail.plot import output_notebook, show
import bokeh
from pprint import pprint
import os
import pandas as pd

In [ ]:
%%capture cap

chrom_list = ( list(range(1,22)) + ['X', 'Y'])

table = (hl.import_table('/content/drive/MyDrive/thesis/Mutual Info Results/HGDP-Pak.txt', impute=True).key_by('Sample name'))
import dask.dataframe as pd
pheno = pd.read_csv("/content/drive/MyDrive/thesis/Mutual Info Results/HGDP-Pak.txt", delimiter='\t')

# due to memory constraints, this pipeline is run on each chromosome one by one separately
for chr in ['Y']:
    try:
        print("\nReading Chromosome", chr)
        mydata = hl.import_vcf('/content/hgdp_wgs.20190516.full.chr'+str(chr)+'.vcf.gz', reference_genome='GRCh38', force_bgz=True)
        mt = mydata.annotate_cols(pheno = table[mydata.s])

        print('\nCount;', mt.count())
        mt2 = mt.filter_cols(mt.pheno['Population name'] != '')
        print('\nTaking Pakistani Population;', mt2.count())

        mt2 = mt2.drop('PL')

        print("\nSplitting into Biallelics;")
        mt3 = hl.split_multi_hts(mt2)
        print("\nWriting")
        mt3.write("/content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split.mt")
        mt3 = hl.read_matrix_table("/content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split.mt")

        print("\nLD Pruning;")
        pruned_variant_table = hl.ld_prune(mt3.GT, r2=0.2, bp_window_size=50000)
        filtered_mt = mt3.filter_rows(hl.is_defined(pruned_variant_table[mt3.row_key]))
        filtered_mt.write("/content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split_ldpruned.mt", overwrite=True)
        os.system("rm -r /content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split.mt")

        filtered_mt = hl.read_matrix_table("/content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split_ldpruned.mt")
        print(filtered_mt.count())

        print("\nExporting Genotypes")
        filtered_mt.GT.export("/content/drive/MyDrive/thesis/HGDP-chr"+str(chr)+"-GT-Samples_split&ldpruned.tsv")

        print("Deleting VCFs")
        os.system('rm hgdp_wgs.20190516.full.chr'+str(chr)+'.vcf.gz.tbi')
        os.system('rm hgdp_wgs.20190516.full.chr'+str(chr)+'.vcf.gz')
        os.system("rm -r /content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split.mt")
        os.system("/content/drive/MyDrive/thesis/hgdp-"+str(chr)+"_split_ldpruned.mt")

        print("Reading")
        df_gt1 = pd.read_csv("/content/drive/MyDrive/thesis/HGDP-chrX-GT-Samples_split&ldpruned.tsv",
                             delimiter='\t',
                             low_memory=False
                             )

        df_gt1['bp'] = df_gt1['locus'] + ':' + df_gt1['alleles']
        del df_gt1['locus']
        del df_gt1['alleles']

        df_gt1 = df_gt1.set_index("bp").T
        print("\nSaving GT csv with new column and transposing")
        df_gt1.fillna(3, inplace=True)
        df_gt1.replace('0/0', 0, inplace=True)
        df_gt1.replace('0/1', 1, inplace=True)
        df_gt1.replace('1/0', 1, inplace=True)
        df_gt1.replace('1/1', 2, inplace=True)
        df_gt1.to_csv("/content/drive/MyDrive/thesis/HGDP Chromosomes Results/HGDP-chr"+str(chr)+"-GT-Samples_split&ldpruned&T.csv")

        print("\nApplying Mutual Info Algorithm")
        df_gt1= pd.read_csv("/content/drive/MyDrive/thesis/HGDP Chromosomes Results/HGDP-chr"+str(chr)+"-GT-Samples_split&ldpruned&T.csv")

        merged_left = pd.merge(left=df_gt1, right=pheno, how='left', left_on='Unnamed: 0', right_on='Sample name')


        x = merged_left.drop(['Sex', 'Unnamed: 0', 'Population name', 'Sample name', 'Superpopulation name',	'Population elastic ID', 'Data collections'], axis=1)
        y = merged_left['Population name']

        from sklearn.feature_selection import mutual_info_classif as MIC
        mi_score = MIC(x,y)
        print(mi_score)

        print("\nSaving outputs")
        mi_scores_df = pd.DataFrame({
            'Columns':x.columns,
            'Scores':mi_score})
        mi_scores_df.sort_values(ascending=False, by='Scores').to_csv("/content/drive/MyDrive/thesis/Mutual Info Results/hgdp-"+str(chr)+"_mi_scores.csv")

        print("\nDone")

    except Exception as e: print(e)


Reading

Applying Mutual Info Algorithm
[0.06308505 0.1248506  0.00060279 ... 0.         0.         0.0913441 ]

Saving outputs

Done


In [ ]:
merged_left

,Unnamed: 0,"chrY:2781761:[""CA"",""C""]","chrY:2781761:[""CAA"",""C""]","chrY:2782347:[""A"",""G""]","chrY:2782439:[""G"",""C""]","chrY:2782986:[""C"",""T""]","chrY:2785100:[""T"",""C""]","chrY:2786162:[""C"",""G""]","chrY:2788215:[""T"",""C""]","chrY:2789135:[""C"",""T""]",...,"chrY:56887316:[""G"",""A""]","chrY:56887463:[""C"",""A""]","chrY:56887503:[""G"",""T""]","chrY:56887583:[""G"",""A""]",Sample name,Sex,Population name,Superpopulation name,Population elastic ID,Data collections
0,HGDP00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,HGDP00001,male,Brahui,Central South Asia (HGDP),BrahuiHGDP,Human Genome Diversity Project
1,HGDP00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,HGDP00003,male,Brahui,Central South Asia (HGDP),BrahuiHGDP,Human Genome Diversity Project
2,HGDP00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,HGDP00005,male,Brahui,Central South Asia (HGDP),BrahuiHGDP,Human Genome Diversity Project
3,HGDP00007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,HGDP00007,male,Brahui,Central South Asia (HGDP),BrahuiHGDP,Human Genome Diversity Project
4,HGDP00009,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,HGDP00009,male,Brahui,Central South Asia (HGDP),BrahuiHGDP,Human Genome Diversity Project
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,HGDP00417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,HGDP00417,male,Burusho,Central South Asia (HGDP),BurushoHGDP,Human Genome Diversity Project
149,HGDP00423,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,HGDP00423,male,Burusho,Central South Asia (HGDP),BurushoHGDP,Human Genome Diversity Project
150,HGDP00433,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,HGDP00433,male,Burusho,Central South Asia (HGDP),BurushoHGDP,Human Genome Diversity Project
151,HGDP00438,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,HGDP00438,male,Burusho,Central South Asia (HGDP),BurushoHGDP,Human Genome Diversity Project
